# Candidate-Sampling with Tensorflow

We investigate some candidate sampling methods such as:
* noise contrastive estimation
* negative sampling
* sampled softmax

Additionally, we also implement hierarchical-softmax.

## 1. Classification

* minimized VGG model
* cifar-100 dataset
* output: softmax / sampled softmax / hierarchical softmax

To download cifar-100 dataset, use this command:
```sh
curl -o cifar-100-binary.tar.gz https://www.cs.toronto.edu/~kriz/cifar-100-binary.tar.gz
tar -xvf cifar-100-binary.tar.gz
```

### 1.1. Model

In [ ]:
import numpy as np
import tensorflow as tf

from cifar_input import build_input

In [ ]:
images, labels = build_input('cifar100', './cifar-100-binary/train.bin', 100, 'train')
drop_rate = tf.placeholder(tf.bool, ())

In [ ]:
def simpleCNN(X, rate, reuse=False):
    X = tf.layers.conv2d(X, 64, [3, 3], padding='same', activation=tf.nn.relu, name='c11', reuse=reuse)
    X = tf.layers.conv2d(X, 64, [3, 3], padding='same', activation=tf.nn.relu, name='c12', reuse=reuse)
    X = tf.layers.max_pooling2d(X, [2, 2], [2, 2])
    
    X = tf.layers.conv2d(X, 128, [3, 3], padding='same', activation=tf.nn.relu, name='c21', reuse=reuse)
    X = tf.layers.conv2d(X, 128, [3, 3], padding='same', activation=tf.nn.relu, name='c22', reuse=reuse)
    X = tf.layers.max_pooling2d(X, [2, 2], [2, 2])
    
    X = tf.layers.conv2d(X, 256, [3, 3], padding='same', activation=tf.nn.relu, name='c31', reuse=reuse)
    X = tf.layers.conv2d(X, 256, [3, 3], padding='same', activation=tf.nn.relu, name='c32', reuse=reuse)
    X = tf.layers.conv2d(X, 256, [3, 3], padding='same', activation=tf.nn.relu, name='c33', reuse=reuse)
    X = tf.layers.max_pooling2d(X, [2, 2], [2, 2])
    
    X = tf.layers.conv2d(X, 512, [3, 3], padding='same', activation=tf.nn.relu, name='c41', reuse=reuse)
    X = tf.layers.conv2d(X, 512, [3, 3], padding='same', activation=tf.nn.relu, name='c42', reuse=reuse)
    X = tf.layers.conv2d(X, 512, [3, 3], padding='same', activation=tf.nn.relu, name='c43', reuse=reuse)
    X = tf.layers.max_pooling2d(X, [2, 2], [2, 2])
    
    X = tf.contrib.layers.flatten(X)
    X = tf.layers.dense(X, 1048, activation=tf.nn.relu, name='d1', reuse=reuse)
    X = tf.layers.dropout(X, rate)
    X = tf.layers.dense(X, 1048, activation=tf.nn.relu, name='d2', reuse=reuse)
    X = tf.layers.dropout(X, rate)
    X = tf.layers.dense(X, 100, name='d3', reuse=reuse)
    
    return X

In [ ]:
logit = simpleCNN(images, drop_rate)

images_tst, labels_tst = build_input('cifar100', './cifar-100-binary/test.bin', 100, 'test')
logit_tst = simpleCNN(images_tst, drop_rate, reuse=True)

### 1.2. Training Type

* softmax
* sampled-softmax
* hierarchical-softmax

#### 1.2.1. Softmax

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=labels))

#### 1.2.2. Sampled-softmax

In [ ]:
l_weights = tf.Variable(tf.truncated_normal([100, 1048],
                                              stddev=1.0 / np.sqrt(1048)))
l_biases = tf.Variable(tf.zeros([100]))


# loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
num_true = 1
num_sampled = 30
num_classes = 100

sampled_values = tf.nn.uniform_candidate_sampler(
          true_classes=tf.reshape(tf.arg_max(labels, 1), [-1,1]),
          num_true=num_true,
          num_sampled=num_sampled,
          unique=True,
          range_max=num_classes)

cost = tf.reduce_mean(
  tf.nn.sampled_softmax_loss(weights=l_weights,
                 biases=l_biases,
                 labels=tf.reshape(tf.arg_max(labels, 1), [-1,1]),
                 inputs=logit.graph.get_operation_by_name('dropout_2/Identity').outputs[0],
                 num_sampled=num_sampled,
                 num_classes=num_classes,
                 sampled_values=sampled_values))
                 
logit = tf.matmul(logit.graph.get_operation_by_name('dropout_2/Identity').outputs[0], tf.transpose(l_weights)) + l_biases
logit_tst = tf.matmul(logit_tst.graph.get_operation_by_name('dropout_4/Identity').outputs[0], tf.transpose(l_weights)) + l_biases

#### 1.2.3. Hierarchical-softmax

For convinience, we use two-layer hierarchical-softmax.

Actually, two-layer approach is best for CIFAR-100, because all training/test labels are uniformly distributed.

---
**Note**

* This Hierarchical-Softmax is not memory-efficient nor gpu-friendly.
* That is the reason why tensorflow-version hierarchical-softmax is not implemented publicly.

In [ ]:
ids = tf.arg_max(labels, 1)
ids_1st = tf.cast(ids // 10, tf.int32)
ids_2nd = tf.cast(ids % 10, tf.int32)

In [ ]:
inputs = logit.graph.get_operation_by_name('dropout_2/Identity').outputs[0]
hs_1st = tf.nn.softmax(tf.layers.dense(inputs, 10, name='hs_1st'))

In [ ]:
p1 = tf.gather(tf.reshape(hs_1st, [-1,1]), ids_1st + tf.range(0, 1000, 10))

p2 = []
for i in range(10):
    matched_nums = tf.reshape(tf.where(tf.equal(ids_1st, i)), [-1])
    matched_ids_2nd = tf.gather(ids_2nd, matched_nums)
    matched_inputs = tf.gather(inputs, matched_nums)
    
    matched_outputs = tf.nn.softmax(tf.layers.dense(matched_inputs, 10, name='hs_2nd_%i' % i))
    matched_ps = tf.gather(tf.reshape(matched_outputs, [-1,1]), matched_ids_2nd + tf.range(0, tf.reduce_prod(tf.shape(matched_outputs)), 10))
    p2.append(matched_ps)
    
p2 = tf.concat(p2, 0)

In [ ]:
p_t = p1 * p2
cost = -tf.reduce_mean(tf.log(p_t))

In [ ]:
logit_1 = tf.nn.softmax(tf.layers.dense(inputs, 10, name='hs_1st', reuse=True))
logit_2 = [tf.nn.softmax(tf.layers.dense(inputs, 10, name='hs_2nd_%i' % i, reuse=True)) for i in range(10)]
logit_2_concat = tf.concat(logit_2, 1)

logit_mul_shape = tf.concat([tf.shape(logit_1), [tf.shape(logit_2)[-1]]], 0)

logit = tf.reshape(tf.expand_dims(logit_1, 2) * 
                   tf.reshape(logit_2_concat, logit_mul_shape), 
                   tf.shape(logit_2_concat))

In [ ]:
inputs_tst = logit.graph.get_operation_by_name('dropout_4/Identity').outputs[0]
logit_tst_1 = tf.nn.softmax(tf.layers.dense(inputs_tst, 10, name='hs_1st', reuse=True))
logit_tst_2 = [tf.nn.softmax(tf.layers.dense(inputs_tst, 10, name='hs_2nd_%i' % i, reuse=True)) for i in range(10)]
logit_tst_2_concat = tf.concat(logit_tst_2, 1)

logit_tst_mul_shape = tf.concat([tf.shape(logit_tst_1), [tf.shape(logit_tst_2)[-1]]], 0)

logit_tst = tf.reshape(tf.expand_dims(logit_tst_1, 2) * 
                   tf.reshape(logit_tst_2_concat, logit_tst_mul_shape), 
                   tf.shape(logit_tst_2_concat))

### 1.3. Training & Test & Logging

In [ ]:
a = tf.placeholder(tf.float32, ())
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.global_variables_initializer()

sess = tf.Session()
tf.train.start_queue_runners(sess)
sess.run(init)

# logging
tf.summary.scalar('learning_rate', a)
tf.summary.scalar('cost', cost)
summaries = tf.summary.merge_all()

summary_writer = tf.summary.FileWriter('./')

In [ ]:
training_epochs = 50000
display_step = 500

for epoch in range(training_epochs+1):
    if epoch < 20000:
        lrn_rate = 0.1
    elif epoch < 30000:
        lrn_rate = 0.01
    elif epoch < 40000:
        lrn_rate = 0.001
    else:
        lrn_rate = 0.0001
        
    _, s, c = sess.run([train, summaries, cost], feed_dict={a: lrn_rate, drop_rate: 0.1})
    summary_writer.add_summary(s, epoch)
    
    print ("Epoch:", '%04d' %(epoch), "cost:", "{:0.9f}".format(c), end='\r')

    if epoch % display_step == 0:
        print()
        rets, anss = [], []
        for i in range(100):
            ret, ans = sess.run([logit_tst, labels_tst], feed_dict={drop_rate: 0.0})
            rets.extend(ret)
            anss.extend(ans)
        precision = np.mean(np.argmax(rets, 1) == np.argmax(anss, 1)) 
        print(precision)
        
        precision_summ = tf.Summary()
        precision_summ.value.add(tag='precision', simple_value=precision)
        summary_writer.add_summary(precision_summ, epoch)

### 1.4. Results

![title](logs/candidate-sampling/precision.png)

## 2. Word2Vec

* codes from https://www.tensorflow.org/tutorials/word2vec
* cost: nce (noise contrastive estimation) / negative sampling

### 2.1. Model

In [ ]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Basic word2vec example."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

#### 2.1. Dataset

In [ ]:
# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

vocabulary = read_data(filename)
print('Data size', len(vocabulary))

In [ ]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

In [ ]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    data_index = 0
    while True:
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window
        batch = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
        span = 2 * skip_window + 1  # [ skip_window target skip_window ]
        buffer = collections.deque(maxlen=span)
        for _ in range(span):
            buffer.append(data[data_index])
            data_index = (data_index + 1) % len(data)
        for i in range(batch_size // num_skips):
            target = skip_window  # target label at the center of the buffer
            targets_to_avoid = [skip_window]
            for j in range(num_skips):
                while target in targets_to_avoid:
                    target = random.randint(0, span - 1)
                targets_to_avoid.append(target)
                batch[i * num_skips + j] = buffer[skip_window]
                labels[i * num_skips + j, 0] = buffer[target]
            buffer.append(data[data_index])
            data_index = (data_index + 1) % len(data)
        # Backtrack a little bit to avoid skipping words in the end of a batch
        yield batch, labels
        data_index = (data_index + len(data) - span) % len(data)
    
batches = generate_batch(batch_size=8, num_skips=2, skip_window=1)
batch, labels = next(batches)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
          '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

In [ ]:
# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

# Input data.
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Look up embeddings for inputs.
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

### 2.2. Training Type
* nce
* negative sampling

In [ ]:
# Uniformly distributed samples
num_true = 1
sampled_values = tf.nn.uniform_candidate_sampler(
          true_classes=tf.cast(train_labels, tf.int64),
          num_true=num_true,
          num_sampled=num_sampled,
          unique=True,
          range_max=vocabulary_size)

#### 2.2.1. NCE

In [ ]:
# Construct the variables for the NCE loss
nce_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                        stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
loss = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights,
                   biases=nce_biases,
                   labels=train_labels,
                   inputs=embed,
                   num_sampled=num_sampled,
                   num_classes=vocabulary_size,
                   sampled_values=sampled_values))

#### 2.2.2. Negative Sampling

In [ ]:
from tensorflow.python.ops.nn_impl import _compute_sampled_logits, _sum_rows

def neg_loss(weights,
             biases,
             labels,
             inputs,
             num_sampled,
             num_classes,
             num_true=1,
             sampled_values=None,
             remove_accidental_hits=False,
             partition_strategy="mod",
             name="neg_loss"):

    logits, labels = _compute_sampled_logits(
        weights=weights,
        biases=biases,
        labels=labels,
        inputs=inputs,
        num_sampled=num_sampled,
        num_classes=num_classes,
        num_true=num_true,
        sampled_values=sampled_values,
        subtract_log_q=False,
        remove_accidental_hits=remove_accidental_hits,
        partition_strategy=partition_strategy,
        name=name)
    sampled_losses = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=labels, logits=logits, name="sampled_losses")
    # sampled_losses is batch_size x {true_loss, sampled_losses...}
    # We sum out true and sampled losses.
    return _sum_rows(sampled_losses)

In [ ]:
# Construct the variables for the NCE loss
neg_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                        stddev=1.0 / math.sqrt(embedding_size)))
neg_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
loss = tf.reduce_mean(
    neg_loss(weights=neg_weights,
                   biases=neg_biases,
                   labels=train_labels,
                   inputs=embed,
                   num_sampled=num_sampled,
                   num_classes=vocabulary_size,
                   sampled_values=sampled_values))

### 2.3. Training

In [ ]:
# Construct the SGD optimizer using a learning rate of 1.0.
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

# Compute the cosine similarity between minibatch examples and all embeddings.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
similarity = tf.matmul(
    valid_embeddings, normalized_embeddings, transpose_b=True)

# Add variable initializer.
init = tf.global_variables_initializer()

In [ ]:
# Step 5: Begin training.
num_steps = 100001
batches = generate_batch(batch_size, num_skips, skip_window)

with tf.Session() as sess:
  # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = next(batches)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

### 2.4. Results

In [ ]:
# Step 6: Visualize the embeddings.
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)

try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    %matplotlib inline
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels)

except ImportError:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')

#### 2.4.1. NCE
![title](logs/candidate-sampling/tsne_nce.png)

#### 2.4.2, Negative Sampling
![title](logs/candidate-sampling/tsne_neg.png)